# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('output_data/CSV/cities.csv')
cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kristiansund,0,NO,1589963947,62,63.11,7.73,280.15,4.10
1,1,Punta Arenas,94,CL,1589963948,92,-53.15,-70.92,272.15,1.50
2,2,Saint-Pierre,20,RE,1589963950,61,-21.34,55.48,298.15,10.30
3,3,Jamestown,1,US,1589963732,50,42.10,-79.24,285.37,2.10
4,4,Mount Isa,98,AU,1589963952,41,-20.73,139.50,299.15,6.70
...,...,...,...,...,...,...,...,...,...,...
555,555,Ola,66,RU,1589964623,63,59.58,151.28,279.38,1.79
556,556,Qingyuan,75,CN,1589964624,83,23.70,113.03,303.71,4.00
557,557,La Santisima Trinidad,0,BO,1589964625,93,-14.83,-64.90,295.33,1.50
558,558,Margate,0,GB,1589964626,87,51.38,1.39,294.82,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Using the Lat and Lng as locations
locations = cities[["Lat", "Lng"]].astype(float)

# Using Humidity
humidity= cities['Humidity'].astype(float)

# Removing Null Cells
cities = cities.dropna()

In [4]:
# Setting the layout for the map
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Setting the fig and adding the center point for zoom of the map
fig = gmaps.figure(layout=figure_layout, center = (42,-42), zoom_level= 3)

# Creating Heat Layer on the map based on Locations and Humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

# Displaying the Fig
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrowing Down the DataFrame based on some criteria:
# 280 F < Max Temp < 300 F
# Wind Speed < 1 mph
# Cloudiness = 0 %
weather_criteria_df = cities [(cities['Max Temp']<300) & (cities['Max Temp']>280) & (cities['Wind Speed']<1.5) & (cities['Cloudiness']==0)]

# Removing Null Cells
weather_criteria_df.dropna()

# Saving the DataFrame to a CSV
weather_criteria_df.to_csv('output_data/CSV/Weather Criteria DF.csv', header = True, index = False)

# Displaying the DataFrame
weather_criteria_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
43,43,Saldanha,0,ZA,1589963999,63,-33.01,17.94,289.15,1.00
135,135,Lebu,0,CL,1589964107,85,-37.62,-73.65,282.90,1.49
173,173,Puerto Quijarro,0,BO,1589964156,90,-17.78,-57.77,294.77,1.12
216,216,Tiznit Province,0,MA,1589964208,29,29.58,-9.50,298.45,0.49
312,312,Santiago del Estero,0,AR,1589964333,83,-27.80,-64.26,290.83,1.49
358,358,Port Shepstone,0,ZA,1589964386,57,-30.74,30.45,294.82,1.16
361,361,Coahuayana Viejo,0,MX,1589964310,62,18.73,-103.68,294.98,1.19
441,441,Las Juntas,0,MX,1589964310,84,19.42,-104.70,292.76,0.80
445,445,Luan,0,CH,1589964493,60,46.36,6.98,291.48,0.45
480,480,Palmares do Sul,0,BR,1589964533,81,-30.26,-50.51,288.52,1.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Creating a new DataFrame based on City, Country, Lat and Lng Columns
hotel_df = weather_criteria_df[['City', 'Country', 'Lat', 'Lng']]

# Setting Lat and Lng as Locations
location = hotel_df[['Lat', 'Lng']]

# Creating empty columns as Hotel Name and Hotel Lat and Lng for showing the accurate Location of the hotel on the map
hotel_df['Hotel Name']= ""
hotel_df['Hotel_Lat']= ""
hotel_df['Hotel_Lng']= ""

# Displaying the DataFrame
hotel_df

<ipython-input-6-bd2d8c719bde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= ""
<ipython-input-6-bd2d8c719bde>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel_Lat']= ""
<ipython-input-6-bd2d8c719bde>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,City,Country,Lat,Lng,Hotel Name,Hotel_Lat,Hotel_Lng
43,Saldanha,ZA,-33.01,17.94,,,
135,Lebu,CL,-37.62,-73.65,,,
173,Puerto Quijarro,BO,-17.78,-57.77,,,
216,Tiznit Province,MA,29.58,-9.50,,,
312,Santiago del Estero,AR,-27.80,-64.26,,,
358,Port Shepstone,ZA,-30.74,30.45,,,
361,Coahuayana Viejo,MX,18.73,-103.68,,,
441,Las Juntas,MX,19.42,-104.70,,,
445,Luan,CH,46.36,6.98,,,
480,Palmares do Sul,BR,-30.26,-50.51,,,


In [7]:
# Setting the base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting the parameters for searching the nearest Hotel in 5000 meter of the Locations
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# uUsing iterrows to iterate through pandas DataFrame
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    # Changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
   
    # Requesting the URL 
    response = requests.get(base_url, params=params).json()
   
    
    results = response['results']

    
    try:
        print(f"Hotel {results[0]['name']} is the first hotel in {row['City']}, {row['Country']}.")
        
        # Filling the Hotel Name, Hotel_Lat and Hotel_Lng columns of the DataFrame with earned Info
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel_Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel_Lng'] = results[0]['geometry']['location']['lng']
   

    # Printing Missing Info when there weren't any result   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("********************")

Hotel Saldanha is the first hotel in Saldanha, ZA.
********************


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel Lebu is the first hotel in Lebu, CL.
********************
Hotel Puerto Quijarro is the first hotel in Puerto Quijarro, BO.
********************
Hotel Ouijjane is the first hotel in Tiznit Province, MA.
********************
Hotel Santiago del Estero is the first hotel in Santiago del Estero, AR.
********************
Hotel Port Shepstone is the first hotel in Port Shepstone, ZA.
********************
Hotel Coahuayana de Hidalgo is the first hotel in Coahuayana Viejo, MX.
********************
Hotel Las Juntas is the first hotel in Las Juntas, MX.
********************
Hotel Aigle is the first hotel in Luan, CH.
********************
Hotel Palmares do Sul is the first hotel in Palmares do Sul, BR.
********************


In [8]:
# Displaying the DataFrame to be show that columns are filled
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel_Lat,Hotel_Lng
43,Saldanha,ZA,-33.01,17.94,Saldanha,-33.0277,17.9176
135,Lebu,CL,-37.62,-73.65,Lebu,-37.6097,-73.6483
173,Puerto Quijarro,BO,-17.78,-57.77,Puerto Quijarro,-17.8026,-57.7762
216,Tiznit Province,MA,29.58,-9.50,Ouijjane,29.6242,-9.53954
312,Santiago del Estero,AR,-27.80,-64.26,Santiago del Estero,-27.7834,-64.2642
358,Port Shepstone,ZA,-30.74,30.45,Port Shepstone,-30.7277,30.4473
361,Coahuayana Viejo,MX,18.73,-103.68,Coahuayana de Hidalgo,18.6928,-103.663
441,Las Juntas,MX,19.42,-104.70,Las Juntas,19.4303,-104.706
445,Luan,CH,46.36,6.98,Aigle,46.319,6.97057
480,Palmares do Sul,BR,-30.26,-50.51,Palmares do Sul,-30.2597,-50.5111


In [9]:
# Removing Null Cells
# As it couldn't find the null cells by Dropna, I had to set the non_values to Float instead of Objects and then replace them and finally use Dropna
nan_value=float("Nan")
hotel_df.replace("",nan_value, inplace = True)
hotel_df.dropna(subset = ['Hotel Name'] , inplace = True)

# Displaying the final DataFrame after removing Null Cells
hotel_df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-9-f47b6dd298a1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(subset = ['Hotel Name'] , inplace = True)


,City,Country,Lat,Lng,Hotel Name,Hotel_Lat,Hotel_Lng
43,Saldanha,ZA,-33.01,17.94,Saldanha,-33.0277,17.9176
135,Lebu,CL,-37.62,-73.65,Lebu,-37.6097,-73.6483
173,Puerto Quijarro,BO,-17.78,-57.77,Puerto Quijarro,-17.8026,-57.7762
216,Tiznit Province,MA,29.58,-9.50,Ouijjane,29.6242,-9.53954
312,Santiago del Estero,AR,-27.80,-64.26,Santiago del Estero,-27.7834,-64.2642
358,Port Shepstone,ZA,-30.74,30.45,Port Shepstone,-30.7277,30.4473
361,Coahuayana Viejo,MX,18.73,-103.68,Coahuayana de Hidalgo,18.6928,-103.663
441,Las Juntas,MX,19.42,-104.70,Las Juntas,19.4303,-104.706
445,Luan,CH,46.36,6.98,Aigle,46.319,6.97057
480,Palmares do Sul,BR,-30.26,-50.51,Palmares do Sul,-30.2597,-50.5111


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name: </dt><dd>{Hotel Name}</dd>
<dt>City: </dt><dd>{City}</dd>
<dt>Country: </dt><dd>{Country}</dd>
</dl>
"""

# Saving the DataFrame to a CSV
hotel_df.to_csv('output_data/CSV/Hotel_DF.csv', header = True, index = False)

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Setting the Hotel_Lat and Hotel_Lng as Locations for accurate Marker Location on the map
locations = hotel_df[["Hotel_Lat", "Hotel_Lng"]]

In [11]:
# Setting the Fig
fig = gmaps.figure(layout=figure_layout, center = (42,-42), zoom_level= 2)

# Creating Markers based on the Locations and with the info box of Hotel Name, Lat and Lng
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Adding marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)


# Displaying Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…